In [1]:
import numpy as np
import pandas
import data as dt
import statsmodels.tsa.api as sm
from statsmodels.tsa.vecm.vecm import VECM

Data import
===========

In [2]:
iidata = dt.load_pandas()

In [3]:
mdata = iidata.data

dates = mdata[['year', 'quarter']].astype(int).astype(str)

quarterly = dates["year"] + "Q" + dates["quarter"]

from statsmodels.tsa.base.datetools import dates_from_str

quarterly = dates_from_str(quarterly)

mdata = mdata[['Dp','R']]

mdata.index = pandas.DatetimeIndex(quarterly)

data = mdata

In [4]:
model = VECM(data)

In [5]:
results = model.fit(2)

fit called
2


In [6]:
results==None

True

In [7]:
type(data)                     # data is what VECM gets. 

pandas.core.frame.DataFrame

LS estimator
============

In [8]:
y_all = data.values.T;   y_all.shape

(2, 107)

In [9]:
diff_lags = 3       # p-1
p = diff_lags + 1   # lags / size of needed presample
K = y_all.shape[0]

In [10]:
Delta_y_all = np.diff(y_all);   Delta_y_all.shape

(2, 106)

In [11]:
y_1_T = y_all[:, p:];   y_1_T.shape

(2, 103)

In [12]:
T = y_1_T.shape[1]

In [13]:
Delta_y_1_T = Delta_y_all[:, p-1:];   Delta_y_1_T.shape

(2, 103)

In [14]:
y_minus1 = y_all[:, p-1:-1];   y_minus1.shape

(2, 103)

In [15]:
Delta_x = np.zeros((diff_lags*K, y_minus1.shape[1]));   Delta_x.shape

(6, 103)

In [16]:
for i in range(Delta_x.shape[1]):
    Delta_x[:, i] = Delta_y_all[:, i+p-2:None if i-1<0 else i-1:-1].T.reshape(K*(p-1))

In [17]:
mat1 = np.hstack((Delta_y_1_T.dot(y_minus1.T), Delta_y_1_T.dot(Delta_x.T)))

In [18]:
B = y_minus1.dot(Delta_x.T)
mat2 = np.linalg.inv(np.vstack( (np.hstack( (y_minus1.dot(y_minus1.T),   B                    ) ),
                                 np.hstack( (B.T,                        Delta_x.dot(Delta_x.T) )) ) ))

In [20]:
estPiGamma = mat1.dot(mat2)
estPiGamma.round(2)

array([[-0.54,  0.06, -0.59,  0.12, -0.71,  0.19, -0.83,  0.02],
       [ 0.13, -0.02, -0.1 ,  0.18, -0.05, -0.04,  0.  ,  0.17]])

In [21]:
Pi_hat = estPiGamma[:, :K]
Pi_hat

array([[-0.54327263,  0.05905934],
       [ 0.12695865, -0.02184917]])

In [22]:
Gamma_hat = estPiGamma[:, K:]
np.round(Gamma_hat, 2)

array([[-0.59,  0.12, -0.71,  0.19, -0.83,  0.02],
       [-0.1 ,  0.18, -0.05, -0.04,  0.  ,  0.17]])

In [23]:
_A = Delta_y_1_T - Pi_hat.dot(y_minus1) - Gamma_hat.dot(Delta_x)
Sigma_u_hat = 1/(y_1_T.shape[1]-K*p) * np.dot(_A,_A.T)
Sigma_u_hat

array([[  3.94865227e-05,  -3.82768919e-06],
       [ -3.82768919e-06,   3.03766974e-05]])

EGLS estimator
==============

In [24]:
r = 1 # r will be given as an function argument. r \in \{0, \ldots, K\}. r=0, r=K?
alpha_hat = Pi_hat[:, :r] # LS Gamma_hat has to be calculated before this line
M = np.identity(y_1_T.shape[1]) - Delta_x.T.dot(np.linalg.inv(Delta_x.dot(Delta_x.T))).dot(Delta_x)
R_0 = Delta_y_1_T.dot(M)
R_1 = y_minus1.dot(M)
R_11 = R_1[:r]
R_12 = R_1[r:]
_alpha_Sigma = alpha_hat.T.dot(np.linalg.inv(Sigma_u_hat))
beta_hhat = np.linalg.inv(_alpha_Sigma.dot(alpha_hat)).dot(_alpha_Sigma).dot(R_0-alpha_hat.dot(R_11)).dot(R_12.T).dot(np.linalg.inv(R_12.dot(R_12.T)))
# ? Gamma_hhat necessary / computed via (Delta_y_1_T - alpha_hat.dot(beta_hhat.T).dot(y_minus1)).dot(Delta_x.dot(np.linalg.inv(np.dot(Delta_x,Delta_x.T))))

ML estimator
============

In [25]:
import scipy

In [26]:
# r, M R_0, R_1 as for the EGLS estimator
S = np.bmat([[Ri.dot(Rj.T)/T for Rj in [R_0, R_1]] for Ri in [R_0, R_1]])
S

matrix([[  3.93591032e-05,  -4.05327169e-06,  -7.62693213e-06,
          -2.03864203e-05],
        [ -4.05327169e-06,   2.84786061e-05,  -3.61793778e-06,
          -4.21339812e-05],
        [ -7.62693213e-06,  -3.61793778e-06,   8.69896387e-05,
           6.71056510e-04],
        [ -2.03864203e-05,  -4.21339812e-05,   6.71056510e-04,
           5.82770122e-03]])

In [30]:
S00 = S[:K, :K]
S01 = S[:K, K:]
S10 = S[K:, :K]
S11 = S[K:, K:]
S11_ = scipy.linalg.sqrtm(S11.I)

v = np.linalg.eig(S11_ * S10 * S00.I * S01 * S11_)[1]
m_beta_tilde = v.T * S11_
m_alpha_tilde = S01 * m_beta_tilde * (m_beta_tilde.T * S11 * m_beta_tilde).I

m_Delta_y_1_T = np.matrix(Delta_y_1_T)
m_y_minus1 = np.matrix(y_minus1)
m_Delta_x = np.matrix(Delta_x)
m_Gamma_tilde = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1) * m_Delta_x.T * (m_Delta_x * m_Delta_x.T).I
m_temp = (m_Delta_y_1_T - m_alpha_tilde*m_beta_tilde.T*m_y_minus1 - m_Gamma_tilde*m_Delta_x)
m_Sigma_u_tilde = m_temp * m_temp.T / T